In [2]:
import tensorflow.keras as keras
keras.__version__

'2.2.4-tf'

# Les réseaux à convolutions

Ces réseaux de neurones comprennent deux nouveaux types de couches :
    
- les couches de convolutions
- les couches de pooling

http://scs.ryerson.ca/~aharley/vis/conv/

# Les couches de convolution

La différence fondamentale entre une couche densément connectée et une couche de convolution est la suivante: 

- les couches denses apprennent des motifs globaux dans leur espace de fonctions en entrée (par exemple, pour un chiffre MNIST, des motifs impliquant tous les pixels), tandis que les couches de convolution apprennent des motifs locaux.

Dans le cas des images, des motifs trouvés dans de petites fenêtres 2D des entrées. 

![image](./images/conv.jpg)


Cette caractéristique clé confère à convnets deux propriétés intéressantes:

- Les modèles qu’ils apprennent sont des invariants de traduction.

- Ils peuvent apprendre les hiérarchies spatiales des modèles

Les convolutions sont définies par deux paramètres clés:

- Taille des patchs extraits des entrées

- Profondeur de la carte de caractéristiques en sortie

![image](./images/conv2.jpg)

# Les couches de pooling

Chaque couche MaxPooling2D permet de réduire la taille des cartes de caractéristiaues. 

C’est le rôle du pooling : sous-échantillonner de manière agressive les cartes de features.

# Un cas concret

Nous allons utiliser notre réseau convnet pour classer les chiffres MNIST.

Les 6 lignes de code ci-dessous vous montrent à quoi ressemble un convnet de base. 


C'est une pile de couches `Conv2D` et` MaxPooling2D`


Il est important de noter qu'un convnet prend en entrée des tenseurs de la forme `(image_height, image_width, image_channels)` 


Dans notre cas, nous allons configurer notre convnet pour traiter des entrées de taille `(28, 28, 1)`, qui est le format des images MNIST. 


Nous faisons cela via l'argument `input_shape = (28, 28, 1)`

In [1]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Using TensorFlow backend.


Voici son architecture :

In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Vous pouvez voir que la sortie de chaque couche `Conv2D` et` MaxPooling2D` est un tenseur 3D de forme `(hauteur, largeur, canaux)`. 


Les dimensions en largeur et en hauteur ont tendance à diminuer à mesure que nous progressons dans le réseau. Le nombre de canaux est contrôlé par le premier argument passé aux couches `Conv2D` (par exemple 32 ou 64).


Nous devons donc aplatir nos sorties 3D sur 1D, puis ajouter quelques couches `Dense` au-dessus:

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

Maintenant, entraînons notre réseau sur les chiffres du MNIST

In [5]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [7]:
train_images[0].shape

(28, 28, 1)

In [11]:
train_labels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [8]:
import imageio

In [9]:
imageio.imsave("./image.png",train_images[0])

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [10]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10, batch_size=64,validation_data=(test_images, test_labels))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 68s 1ms/step - loss: 0.5621 - acc: 0.7928 - val_loss: 0.4628 - val_acc: 0.8183
Epoch 2/10
60000/60000 [==============================] - 70s 1ms/step - loss: 0.3412 - acc: 0.8761 - val_loss: 0.3428 - val_acc: 0.8788
Epoch 3/10
60000/60000 [==============================] - 71s 1ms/step - loss: 0.2871 - acc: 0.8947 - val_loss: 0.2986 - val_acc: 0.8878
Epoch 4/10
60000/60000 [==============================] - 70s 1ms/step - loss: 0.2554 - acc: 0.9059 - val_loss: 0.2824 - val_acc: 0.9004
Epoch 5/10
60000/60000 [==============================] - 71s 1ms/step - loss: 0.2322 - acc: 0.9149 - val_loss: 0.2915 - val_acc: 0.8959
Epoch 6/10
60000/60000 [==============================] - 69s 1ms/step - loss: 0.2131 - acc: 0.9224 - val_loss: 0.2759 - val_acc: 0.9018
Epoch 7/10
60000/60000 [==============================] - 69s 1ms/step - loss: 0.1965 - acc: 0.9280 - val_loss: 0.2853 - val_acc

In [13]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 3s 293us/step


In [14]:
test_acc

0.8844